# Notebook 4.3 - Actors curation: delete actors not associated to any item

This notebook helps moderators to curate actors with no items in the SSH Open Marketplace. 

This notebook is composed of n sections:

0. Requirements to run this notebook
1. Get all actors from the MP dataset 
2. Find actors never associated to any item
3. Delete actors never associated to any item

### *Tests not completed!* 


## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [1]:
import pandas as pd #to manage dataframes

import numpy as np #to manage json objects
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

In [2]:
mpdata = mpd()
utils=hel.Util()
check=eva.URLCheck()

## 1. Get actors

In [3]:
df_actors_flat =mpdata.getMPItems ("actors", False)

Remove duplicates from the local copy, if any

In [5]:
df_actors_flat['norm_name']=df_actors_flat['name'].apply(lambda y: (' ').join(y.split()) if type(y)==str else y)
df_actors_flat_no_duplicates=df_actors_flat.sort_values('name').drop_duplicates(subset=['norm_name'], keep='first', ignore_index=True)

print(f'Number of actors: {df_actors_flat_no_duplicates.shape[0]}')

Number of actors: 7051


In [6]:
df_contrib=utils.getContributors()
df_no=df_actors_flat_no_duplicates.merge(df_contrib['actor.id'], left_on='id', right_on='actor.id', how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
#df_no.sort_values('name').head()

Loading Actors for toolsandservices
Loading Actors for publications
Loading Actors for trainingmaterials
Loading Actors for workflows
Loading Actors for datasets
keyword data is not present, skip...
concepts data is not present, skip...
sshockeywords data is not present, skip...
searchsshockeywords data is not present, skip...


In [16]:
print(f'Number of actors *not currently* associated to any item in the dataset: {df_no.shape[0]}, (some of them could have been associated to an item previously)')

Number of actors *not currently* associated to any item in the dataset: 1097, (some of them could have been associated to an item previously)


### 2 Find actors never associated to any item
The code below search for items never associated to any item. To found these actors it uses the api entry:

    /api/actors/{id}?items=true

and identify those actors having an empty list of associated items.

In [10]:
df_actors_ei=pd.DataFrame()
tot=0
print ('Start...')
for mytindex, mytrow in df_no.iterrows():
    actitems=mpdata.getItemsforActor(str(mytrow.id).replace('.0',''))
    if (actitems.empty):
        tot=tot+1
        if (tot % 100 ==0):
            print (f'found {tot} actors,')
        #print (f'{mytrow.id}, {mytrow["norm_name"]}')
        new_df = pd.DataFrame([mytrow])
        df_actors_ei=pd.concat([df_actors_ei, new_df])
       
print(f'Total number of actors: {tot}')

Start...
Found 100 actors...
Found 200 actors...
Found 300 actors...
Found 400 actors...
Found 500 actors...
Found 600 actors...
Found 700 actors...
Total number of actors: 770


In [ ]:
#df_actors_ei.to_pickle('data/actors_no_items.pickle')

In [11]:
df_actors_ei.sort_values('id').head(10)

,id,name,externalIds,website,affiliations,norm_name,actor.id,_merge
7095,63.0,Microsoft,[],NaN,[],Microsoft,NaN,left_only
10067,74.0,UCREL,[],NaN,[],UCREL,NaN,left_only
9711,345.0,Texifter,[],NaN,[],Texifter,NaN,left_only
8259,354.0,RStudio,[],NaN,[],RStudio,NaN,left_only
4773,663.0,"John B. Smith, Pennsylvania State University",[],NaN,[],"John B. Smith, Pennsylvania State University",NaN,left_only
6025,876.0,"MIT Libraries, MIT CSAIL, Zepheira",[{'identifierService': {'code': 'SourceActorId...,http://www.simile-widgets.org/,[],"MIT Libraries, MIT CSAIL, Zepheira",NaN,left_only
1038,957.0,"Branchfire, Inc",[],NaN,[],"Branchfire, Inc",NaN,left_only
3548,960.0,Geowise Limited,[],NaN,[],Geowise Limited,NaN,left_only
261,963.0,Alistair Baron (UCREL),[],http://www.comp.lancs.ac.uk/~barona/home/,[],Alistair Baron (UCREL),NaN,left_only
8019,964.0,Performant Software Solutions LLC,[],NaN,[],Performant Software Solutions LLC,NaN,left_only


### 3 Delete actors never associated to any item

In [14]:
for niindex,nirow in df_actors_ei.iterrows():
    mpdata.deleteItem('actors', str(nirow.id).replace('.0',''))

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3318
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3317
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3311
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3312
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3316
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3310
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3319
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3313
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1150
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3811
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//9134
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3240
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3207
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1190
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1678
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1662
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4006
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6619
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6610
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1550
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//992
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1390
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8748
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//7834
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1138
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//7767
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1216
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//9272
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1649
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//7182
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5666
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8670
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4485
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5660
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//977
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1686
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1628
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5787
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3832
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6173
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac

...

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6715
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8265
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1217
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//9278
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1232
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6784
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6791
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//7639
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//970
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//2916
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8649
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1544
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4092
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1558
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1648
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1654
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4440
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4646
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1009
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4968
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//2928
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5237
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8218
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5623
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8928
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5234
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8224
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8003
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//1010
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6065
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3626
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6288
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a

URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5960
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//4655
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3605
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//6525
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//3648
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//8024
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5867
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/actors//5875
...not executed, running in DEBUG mode.
URL: https://sshoc-marketplace-api-stage.acdh-dev.oeaw.a